In [43]:
import sys
import os
import pandas as pd
import spacy

In [12]:
%load_ext autoreload
%autoreload 2
PATH = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(PATH)
from skills_matcher.scripts.spacy_model import json_to_train_data, split_data, make_spacy_model, load_results

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
df = pd.read_csv(PATH + '/data/cleaned_data.csv')
text= df['description'][20]
text2 = df['description'][2000]
text2

"[' Ensuring customers are at the heart of everything we do, making sure our expertise and medicines are easily accessed, providing patient services, dispensing, and giving healthcare advice. Leading, motivating, training and developing your colleagues. Keeping the both the customer and the business safe and legal. Building working relationships with Primary Care Networks and GP surgeries Ensuring payment claims are accurate for anything covered by the NHS. Using our modern planning systems to ensure we have the right number of colleagues and pharmacists on duty to meet customer needs ', ' A Pharmacy Degree A passion for leading and empowering a team. Membership of the General Pharmaceutical Council (GPhC) Hepatitis B immunisation (or be prepared to be immunised after starting) Pharmacists working for Tesco will have to complete a DBS check (organised by Tesco) prior to commencement of employment. ', ' Annual payment of your GPhC fee. A structured training plan for Pharmacy Leadership,

In [16]:
df.head(3)

,Unnamed: 0,ISCO,major_job,job,position,location,description
0,0,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...
1,1,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Geneva,[' Be in charge of the development of applicat...
2,2,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,Geneva,[' Contribute to the maintenance and developme...


In [18]:
training_data = json_to_train_data(PATH + '/notebooks/annotation_data.json')
X_train, X_val = split_data(training_data, val_split=0.2)
spacy_train = make_spacy_model(X_train)
spacy_train.to_disk('train.spacy')
spacy_val = make_spacy_model(X_val)
spacy_val.to_disk('val.spacy')

100%|██████████████████████████████████████████| 58/58 [00:00<00:00, 260.94it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|██████████████████████████████████████████| 15/15 [00:00<00:00, 311.98it/s]

Skipping entity
Skipping entity


In [23]:
 ! python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [24]:
! python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-12-03 16:34:21,407] [INFO] Set up nlp object from config
[2021-12-03 16:34:21,413] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-12-03 16:34:21,416] [INFO] Created vocabulary
[2021-12-03 16:34:21,418] [INFO] Finished initializing nlp object
[2021-12-03 16:34:22,199] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     66.89    0.00    0.00    0.00    0.00
  3     200       9998.70   3642.38   24.21   25.69   22.89    0.24
  7     400       5097.39   2184.25   67.14   67.74   66.55    0.67
 10     

In [162]:
text= df['description'][1]

test = '''
Minimum of Bachelor’s or Master’s Degree in Computer Science, Engineering, or similar. 
Strong interest in data management, processing and reporting, including modern architectures and techniques.
Strong coding skills and computer science fundamentals.
Knowledge and experience of statistical techniques and concepts.
Knowledge in cloud technology is a plus.
Knowledge of Linux, Oracle and Tableau/Power BI to manipulate data.
Creative problem solving and critical thinking skills, aiming to improve the existing processes.
Good communication skills to liaise with different interface teams.
Experience working with databases and solving complex business problems with high-volume data.
Flexible to adapt in highly dynamic environments.'''

entities = load_results(PATH + '/notebooks/output/model-best', test)

In [116]:
# string matching

skills = ['Power BI', 'Python']

job_descritions = [_ for _ in df['description']]

In [51]:
nlp = spacy.load(PATH + '/notebooks/output/model-best')

doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

2-3 weeks at 319 331 MIN_EXP
Physics 680 687 KNOWLEDGE
Electrical Engineering 689 711 KNOWLEDGE
2-3 weeks at 1026 1038 MIN_EXP


In [89]:
for token in doc[:3]:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

[ True 20.038563 True
' True 19.297134 True
Act True 20.415789 True


In [168]:
nlp_manual = spacy.load('en_core_web_sm')

doc_manual = nlp_manual(text)

for t in doc_manual[2:5]:
    print(t, t.pos_, t.dep_, t.shape_, t.lemma_, t.tag_)
    
# get POS / Part of Speech

def give_skill(text):
    doc_manual = nlp_manual(text)
    skills = []
    for t in doc_manual:
        if t.lower_ in ['python', 'r', ]:
            if t.pos != 'VERB':
                skills.append(t)
    return skills

give_skill(text)

Be AUX ROOT Xx be VB
in ADP prep xx in IN
charge NOUN pobj xxxx charge NN


[Python, Python]

In [160]:
doc_manual

[' You have an academic degree in physics or engineering with profound knowledge of applied accelerator physics You have a demonstrated ability to compute beam optics and other parameters relevant to beam performance You have experience in experimental techniques and data analysis in the context of beam measurements You enjoy working in a diverse team and communicate efficiently in English. If you do not speak German, you are willing to strive for at least a basic level in that language ']

In [131]:
# get token from a text
print ([token for token in doc])

[[, ', Act, as, an, operational, engineer, and, build, a, complete, and, broad, understanding, of, multi, -, beam, mask, writers, ., Operation, ,, calibration, and, optimization, of, the, electron, beam, ., Data, analysis, and, tool, status, evaluation, ., Implementation, and, documentation, of, experiments, at, the, tool, ., Fly, around, the, world, to, customer, factories, for, 2, -, 3, weeks, at, a, time, to, recalibrate, customer, production, tools, ., Be, able, to, work, independently, to, correlate, trends, across, the, fleet, of, deployed, tools, ., Internal, and, external, communication, about, the, progress, of, tool, optimization, ., Conduct, trainings, on, tool, operation, and, calibration, ., On, -, call, service, for, troubleshooting, ., ', ,, ', A, Bachelor\, 's, Degree, (, or, equivalent, ), in, :, Physics, ,, Electrical, Engineering, ,, Process, Engineering, or, a, related, field, Very, good, spoken, and, written, English, ., Experience, in, using, office, programs, and

In [62]:
doc1 = nlp(text)
doc2 = nlp(text2)

# Similarity of two documents
print(doc1.similarity(doc2))


0.9944898873664768


/var/folders/sy/7155ny0j7zd0yvnmgd8twpxm0000gn/T/ipykernel_36362/3612073840.py:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print(doc1.similarity(doc2))


In [76]:
# Similarity of tokens and spans
test_token1 = doc1[2:6]
test_token2 = doc1[31:35]

test_token3 = doc1[2:6]
test_token4 = doc1[31:35]

print(f' The similarity between the tokens “{test_token1}“ and "{test_token2}" is: {test_token1.similarity(test_token2)}')
print(f' The similarity between the tokens “{test_token3}“ and "{test_token4}" is: {test_token1.similarity(test_token2)}')

 The similarity between the tokens “Act as an operational“ and "Data analysis and tool" is: 0.6857989430427551


/var/folders/sy/7155ny0j7zd0yvnmgd8twpxm0000gn/T/ipykernel_36362/792070772.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print(f' The similarity between the tokens “{test_token1}“ and "{test_token2}" is: {test_token1.similarity(test_token2)}')


In [82]:
doc1.text

'[\' Act as an operational engineer and build a complete and broad understanding of multi-beam mask writers. Operation, calibration and optimization of the electron beam. Data analysis and tool status evaluation. Implementation and documentation of experiments at the tool. Fly around the world to customer factories for 2-3 weeks at a time to recalibrate customer production tools. Be able to work independently to correlate trends across the fleet of deployed tools. Internal and external communication about the progress of tool optimization. Conduct trainings on tool operation and calibration. On-call service for troubleshooting. \', \' A Bachelor\\\'s Degree (or equivalent) in: Physics, Electrical Engineering, Process Engineering or a related field Very good spoken and written English. Experience in using office programs and Unix (Linux) Python and Jira experience is preferred. Deliberate and organized work style with a "hands on" mentality Willingness to be deployed abroad for an avera